In [ ]:
import pandas as pd 
import torch
import sentencepiece as spm
import ast
from torch.nn.utils.rnn import pad_sequence
from sklearn.model_selection import train_test_split
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from copy import deepcopy as dc

In [134]:
df = pd.read_csv("C:\Emotion Classification\\traning\in_model_data\data.csv", sep=',')
df.sample(2)

<>:1: SyntaxWarning: invalid escape sequence '\E'
<>:1: SyntaxWarning: invalid escape sequence '\E'
C:\Users\illya\AppData\Local\Temp\ipykernel_15744\2144087589.py:1: SyntaxWarning: invalid escape sequence '\E'
  df = pd.read_csv("C:\Emotion Classification\\traning\in_model_data\data.csv", sep=',')


,text,example_very_unclear,admiration,amusement,anger,annoyance,approval,caring,confusion,curiosity,...,love,nervousness,optimism,pride,realization,relief,remorse,sadness,surprise,neutral
26201,"[41470, 0, 115, 69, 967, 332, 395, 131, 25202,...",0,0,0,0,0,1,0,0,0,...,1,0,0,0,0,0,0,0,0,0
50582,"[41470, 0, 2658, 41477, 825, 21524, 41488]",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
df['text'] = df['text'].apply(ast.literal_eval)
list_of_lists = df['text'].apply(lambda x: [int(i) for i in x])

In [136]:
type(df['text'][0][0])

int

In [137]:
tensor_list = [torch.tensor(seq, dtype=torch.long) for seq in list_of_lists]
X = pad_sequence(tensor_list, batch_first=True, padding_value=0)
y = df.drop(columns=['text'], axis=1)

In [138]:
y = torch.tensor(y.values, dtype=torch.long)

In [139]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [140]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

(torch.Size([56000, 142]),
 torch.Size([56000, 29]),
 torch.Size([14000, 142]),
 torch.Size([14000, 29]))

In [144]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Використовується пристрій: {device}")

print(f"Розмір тренувальних даних: {X_train.shape}")
print(f"Розмір тренувальних міток: {y_train.shape}")
print(f"Мінімальне значення в X_train: {X_train.min().item()}")
print(f"Максимальне значення в X_train: {X_train.max().item()}")
print(f"Тип даних X_train: {X_train.dtype}")
print(f"Тип даних y_train: {y_train.dtype}")
print()

class GeomotionLSTM(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim, num_layers, output_dim, dropout=0.3):
        super(GeomotionLSTM, self).__init__()
        
        # Embedding шар для перетворення індексів на вектори
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        
        # LSTM шари
        self.lstm = nn.LSTM(
            input_size=embedding_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0,
            bidirectional=False
        )
        
        # Dropout для регуляризації
        self.dropout = nn.Dropout(dropout)
        
        # Повністю підключений шар для класифікації
        self.fc = nn.Linear(hidden_dim, output_dim)
        
        # Sigmoid для багатокласової класифікації
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        # x shape: (batch_size, seq_len)
        
        # Embedding
        embedded = self.embedding(x)  # (batch_size, seq_len, embedding_dim)
        
        # LSTM
        lstm_out, (hidden, cell) = self.lstm(embedded)
        
        # Використовуємо останній вихід LSTM
        last_output = lstm_out[:, -1, :]  # (batch_size, hidden_dim)
        
        # Dropout
        dropped = self.dropout(last_output)
        
        # Класифікація
        output = self.fc(dropped)  # (batch_size, output_dim)
        output = self.sigmoid(output)
        
        return output

# Визначення реального розміру словника з ваших даних
max_index = int(X_train.max().item())
VOCAB_SIZE = max_index + 1  # +1 тому що індексація з 0
print(f"Максимальний індекс: {max_index}")
print(f"Розмір словника: {VOCAB_SIZE}")

# Опційно: обрізання великих індексів якщо потрібно
# Розкоментуйте якщо VOCAB_SIZE занадто великий
# MAX_VOCAB_SIZE = 50000
# if VOCAB_SIZE > MAX_VOCAB_SIZE:
#     print(f"Обрізання словника до {MAX_VOCAB_SIZE}")
#     X_train = torch.clamp(X_train, max=MAX_VOCAB_SIZE-1)
#     X_test = torch.clamp(X_test, max=MAX_VOCAB_SIZE-1)
#     VOCAB_SIZE = MAX_VOCAB_SIZE

print(f"Фінальний розмір словника: {VOCAB_SIZE}")

# Гіперпараметри моделі
EMBEDDING_DIM = 128
HIDDEN_DIM = 256
NUM_LAYERS = 2
OUTPUT_DIM = 29  # 29 класів згідно з вашими даними
DROPOUT = 0.3

# Параметри тренування
BATCH_SIZE = 64
LEARNING_RATE = 0.001
NUM_EPOCHS = 5

# Створення моделі
model = GeomotionLSTM(
    vocab_size=VOCAB_SIZE,
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    num_layers=NUM_LAYERS,
    output_dim=OUTPUT_DIM,
    dropout=DROPOUT
).to(device)

print(f"Модель створена з {sum(p.numel() for p in model.parameters())} параметрами")

# Створення DataLoader'ів
train_dataset = TensorDataset(X_train, y_train)
test_dataset = TensorDataset(X_test, y_test)

train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Функція втрат і оптимізатор
criterion = nn.BCELoss()  # Binary Cross Entropy для багатокласової класифікації
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE, weight_decay=1e-5)

# Scheduler для зменшення learning rate
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

# Функція для обчислення accuracy
def calculate_accuracy(outputs, targets, threshold=0.5):
    predicted = (outputs > threshold).float()
    correct = (predicted == targets).float()
    accuracy = correct.sum() / (correct.size(0) * correct.size(1))
    return accuracy.item()

# Списки для збереження метрик
train_losses = []
train_accuracies = []
val_losses = []
val_accuracies = []

print("Початок тренування...")

for epoch in range(NUM_EPOCHS):
    # Тренування
    model.train()
    train_loss = 0.0
    train_acc = 0.0
    
    for batch_idx, (data, targets) in enumerate(train_loader):
        data, targets = data.to(device), targets.to(device).float()
        
        # Обнулення градієнтів
        optimizer.zero_grad()
        
        # Forward pass
        outputs = model(data)
        loss = criterion(outputs, targets)
        
        # Backward pass
        loss.backward()
        
        # Gradient clipping для стабільності
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
        
        # Оновлення параметрів
        optimizer.step()
        
        # Збереження метрик
        train_loss += loss.item()
        train_acc += calculate_accuracy(outputs, targets)
        
        # Виведення прогресу кожні 100 батчів
        if (batch_idx + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{NUM_EPOCHS}], Batch [{batch_idx+1}/{len(train_loader)}], '
                  f'Loss: {loss.item():.4f}')
    
    # Валідація
    model.eval()
    val_loss = 0.0
    val_acc = 0.0
    
    with torch.no_grad():
        for data, targets in test_loader:
            data, targets = data.to(device), targets.to(device).float()
            
            outputs = model(data)
            loss = criterion(outputs, targets)
            
            val_loss += loss.item()
            val_acc += calculate_accuracy(outputs, targets)
    
    # Обчислення середніх метрик
    avg_train_loss = train_loss / len(train_loader)
    avg_train_acc = train_acc / len(train_loader)
    avg_val_loss = val_loss / len(test_loader)
    avg_val_acc = val_acc / len(test_loader)
    
    # Збереження метрик
    train_losses.append(avg_train_loss)
    train_accuracies.append(avg_train_acc)
    val_losses.append(avg_val_loss)
    val_accuracies.append(avg_val_acc)
    
    # Оновлення learning rate
    scheduler.step()
    
    print(f'Epoch [{epoch+1}/{NUM_EPOCHS}]:')
    print(f'  Train Loss: {avg_train_loss:.4f}, Train Acc: {avg_train_acc:.4f}')
    print(f'  Val Loss: {avg_val_loss:.4f}, Val Acc: {avg_val_acc:.4f}')
    print(f'  Learning Rate: {optimizer.param_groups[0]["lr"]:.6f}')
    print('-' * 60)

# Збереження моделі
torch.save({
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'train_losses': train_losses,
    'train_accuracies': train_accuracies,
    'val_losses': val_losses,
    'val_accuracies': val_accuracies,
    'hyperparameters': {
        'vocab_size': VOCAB_SIZE,
        'embedding_dim': EMBEDDING_DIM,
        'hidden_dim': HIDDEN_DIM,
        'num_layers': NUM_LAYERS,
        'output_dim': OUTPUT_DIM,
        'dropout': DROPOUT
    }
}, 'geomotion_lstm_model.pth')

print("Модель збережена!")

class_accuracies = []
for i in range(OUTPUT_DIM):
    acc = accuracy_score(all_targets[:, i], all_predictions[:, i])
    class_accuracies.append(acc)
    print(f'Клас {i}: Accuracy = {acc:.4f}')


Використовується пристрій: cpu
Розмір тренувальних даних: torch.Size([56000, 142])
Розмір тренувальних міток: torch.Size([56000, 29])
Мінімальне значення в X_train: 0
Максимальне значення в X_train: 41991
Тип даних X_train: torch.int64
Тип даних y_train: torch.int64

Максимальний індекс: 41991
Розмір словника: 41992
Фінальний розмір словника: 41992
Модель створена з 6304029 параметрами
Початок тренування...
Epoch [1/5], Batch [100/875], Loss: 0.1628
Epoch [1/5], Batch [200/875], Loss: 0.1508
Epoch [1/5], Batch [300/875], Loss: 0.1374
Epoch [1/5], Batch [400/875], Loss: 0.1613
Epoch [1/5], Batch [500/875], Loss: 0.1571
Epoch [1/5], Batch [600/875], Loss: 0.1766
Epoch [1/5], Batch [700/875], Loss: 0.1405
Epoch [1/5], Batch [800/875], Loss: 0.1559
Epoch [1/5]:
  Train Loss: 0.1608, Train Acc: 0.9577
  Val Loss: 0.1557, Val Acc: 0.9586
  Learning Rate: 0.001000
------------------------------------------------------------
Epoch [2/5], Batch [100/875], Loss: 0.1527
Epoch [2/5], Batch [200/87

NameError: name 'all_targets' is not defined